In [1]:
%load_ext lab_black
import os
from bs4 import BeautifulSoup as bs
import requests
import time
from splinter import Browser
from selenium import webdriver
import pandas as pd

In [2]:
url = "https://mars.nasa.gov/#news_and_events"

In [3]:
r = requests.get(url)

In [4]:
html = r.text

In [5]:
soup = bs(html, "html.parser")

In [6]:
news_title = (soup.find("div", class_="list_text")).find("h3").text
print(news_title)


A Year of Surprising Science From NASA's InSight Mars Mission



In [7]:
# # # BONUS: The news_p was something that didn't come through because of page loading. You might have to try this with Selenium or Splinter

# url = "https://mars.nasa.gov/#news_and_events"
# driver = webdriver.Firefox()
# driver.get(url)
# time.sleep(1)


# elem = driver.find_element_by_partial_link_text(f'{news_title}')
# elem.click()

# elem = browser.find_element_by_id("full_image")
# elem.click()
# browser.find_element_by_link_text("more info")
# #new_url = elem.get_attribute("href")
# print(new_url)


# browser = webdriver.Firefox()
# browser.get(url)

# browser.find_element_by_id("full_image")

In [8]:
# Connect to a url to grab NASA's featured Mars image

# Use selenium to navegate to the page with the image
driver = webdriver.Firefox()
url_image = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
driver.get(url_image)
time.sleep(1)

elem = driver.find_element_by_id("full_image")
elem.click()
time.sleep(1)

# Navigate to "more info" web site, where the full image URL is
elem = driver.find_element_by_link_text("more info")
elem.click()
time.sleep(1)

# Get the "more info" page URL to parse and find the image in full resolution
url_more_info = driver.current_url

# Use requests and soup to return the image url
r = requests.get(url_more_info)
html = r.text
soup = bs(html, "html.parser")
featured_image_url = (
    "https://www.jpl.nasa.gov" + soup.find("figure", class_="lede").a["href"]
)
print(featured_image_url)

# Close the browser
driver.close()

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA07137_hires.jpg


In [9]:
# Mars Weather
# Visit https://twitter.com/marswxreport?lang=en and scrape the latest Mars weather tweet from the page.
# Save the tweet text for the weather report as a variable called `mars_weather`.

# Connect to url for Mars Weather Twitter page

url_twitter = "https://twitter.com/marswxreport?lang=en"

# Create a Beautiful Soup object
r = requests.get(url_twitter)
html = r.text
soup = bs(html, "html.parser")
mars_weather = soup.find_all("div", class_="js-tweet-text-container")

# find the first actual weather tweet

mars_weather = mars_weather[0].text

print(f"The weather today in Mars is: {mars_weather}")

The weather today in Mars is: 
InSight sol 444 (2020-02-25) low -93.8ºC (-136.8ºF) high -12.0ºC (10.5ºF)
winds from the SSW at 6.2 m/s (13.9 mph) gusting to 21.2 m/s (47.4 mph)
pressure at 6.30 hPapic.twitter.com/UeOmoDjhf3



In [11]:
## Mars Facts - use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
#  Use Pandas to convert the data to a HTML table string.

url_facts = "http://space-facts.com/mars/"
mars_facts = pd.read_html(url_facts)



ImportError: lxml not found, please install it

In [ ]:
### Mars Hemispheres

# set url and the driver
driver = webdriver.Firefox()
url_hem = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
driver.get(url_hem)

# scrape page into Soup
r = requests.get(url_hem)
html = r.text
soup = bs(html, "html.parser")


# Scrape the images of each hemisphere
hemisphere_image_urls = []
hemispheres = ["Cerberus", "Schiaparelli", "Syrtis Major", "Valles Marineris"]
# find hemisphere titles
# hemispheres = soup.find_all("h3")
# print(hemispheres)

In [ ]:
# Loop through the four pages containing the information
for hemisphere in hemispheres:
    h = driver.find_elements_by_partial_link_text(hemisphere)
    h.click()
    time.sleep(1)
    #     html = driver.html
    #     soup = bs(html, 'html.parser')
    #     title = soup.find('h3', class_='title').text
    #     img_url = 'https://web.archive.org/' + soup.find('img', class_='wide-image')['src']
    #     hemisphere_image_urls.append({'title': title, 'img_url': img_url})
    #     driver.back()
    #     time.sleep(1)
    print(h)